## Library

In [2]:
import requests
import random
import pandas as pd
import time
import math
from copy import deepcopy
from statistics import mean

## Notebook

#### Requisição

In [3]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

#### Tratamento de Dados

In [4]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                    if (key in [product for product in json[i] if ('prod' in product and (int(product[5]) < 9 and len(product) < 7))]):
                        json[i][key] = {'qnt':[round(qnt)], 'price':None}
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None}
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

#### Gerar valores aleatórios

In [5]:
def generate_random_price(n_min=1.0, n_max=100.0) -> float:
    return round(random.uniform(n_min, n_max), 2)

#### Precificação

##### Função logistica -> A mesma postada no doc pelo professor

In [28]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / (1 + math.e ** -v))

##### Função para calcular a variação -> A mesma postada no doc pelo professor

In [27]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

##### Função que calcula o preço que será utilizado na semana seguinte

In [14]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['next_week_price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

##### Função que atualiza o banco de dados (onde consta a quantidade de vendas da última semana e o preço dos produtos) e aplica o preço nos produtos do JSON

In [15]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None
                    product_db[key]['next_week_price'] = product_db[key]['price']              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['next_week_price'])
                    product_db[key]['price'] = product_db[key]['next_week_price']
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

#### Main

In [30]:
product_database = {}
weeks = 25

for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    json_db = pd.DataFrame(json)
    product_database_db = pd.DataFrame(product_database)

    print(f'\nSemana {week + 1}:\n')
    print('Json:')
    display(json_db)
    print('\nproduct_database:')
    display(product_database_db)


Semana 1:

Json:


,id,date,prod_0,prod_2,prod_3,prod_4,prod_5,prod_6,prod_8,prod_9,prod_10,prod_15,prod_1,prod_7,prod_11
0,labp6aw8rr4o8gjaq2o,Thu Oct 24 08:49:40 2024,"{'qnt': [9], 'price': 47.77}","{'qnt': [25], 'price': 48.35}","{'qnt': [16], 'price': 5.71}","{'qnt': [43], 'price': 73.05}","{'qnt': [1], 'price': 38.61}","{'qnt': [17], 'price': 60.61}","{'qnt': [430], 'price': 57.28}","{'qnt': [809.6457464313798], 'price': 39.7}","{'qnt': [466.3152960854334], 'price': 15.03}","{'qnt': [1850.186334798945], 'price': 41.48}",NaN,NaN,NaN
1,labp6aw889pawmuo00r,Wed Oct 23 17:43:38 2024,"{'qnt': [22], 'price': 47.77}",NaN,NaN,"{'qnt': [42], 'price': 73.05}","{'qnt': [45], 'price': 38.61}",NaN,"{'qnt': [402], 'price': 57.28}",NaN,"{'qnt': [893.6069315590282], 'price': 15.03}","{'qnt': [2426.738429691921], 'price': 41.48}","{'qnt': [11], 'price': 97.67}","{'qnt': [28], 'price': 99.45}","{'qnt': [907.3229546861892], 'price': 68.67}"
2,labp6aw8nrpt0lh1m7,Thu Oct 24 07:48:38 2024,NaN,NaN,NaN,"{'qnt': [42], 'price': 73.05}","{'qnt': [5], 'price': 38.61}",NaN,"{'qnt': [764], 'price': 57.28}","{'qnt': [28.57175341014382], 'price': 39.7}","{'qnt': [1081.347077254213], 'price': 15.03}","{'qnt': [2032.113666362543], 'price': 41.48}",NaN,"{'qnt': [26], 'price': 99.45}","{'qnt': [398.184360157671], 'price': 68.67}"
3,labp6aw88fvz2h3ul5d,Wed Oct 23 20:57:57 2024,"{'qnt': [48], 'price': 47.77}","{'qnt': [45], 'price': 48.35}","{'qnt': [44], 'price': 5.71}",NaN,"{'qnt': [37], 'price': 38.61}",NaN,"{'qnt': [880], 'price': 57.28}","{'qnt': [1081.1713104023], 'price': 39.7}",NaN,"{'qnt': [2458.1541685178045], 'price': 41.48}","{'qnt': [6], 'price': 97.67}","{'qnt': [4], 'price': 99.45}",NaN
4,labp6aw8kbz51wlrqa,Wed Oct 23 16:23:50 2024,NaN,NaN,NaN,"{'qnt': [7], 'price': 73.05}",NaN,"{'qnt': [46], 'price': 60.61}","{'qnt': [912], 'price': 57.28}","{'qnt': [507.87881065098145], 'price': 39.7}",NaN,"{'qnt': [2359.8538526811826], 'price': 41.48}","{'qnt': [15], 'price': 97.67}","{'qnt': [21], 'price': 99.45}","{'qnt': [444.15808982867486], 'price': 68.67}"
5,labp6aw8a5nnljanx3a,Wed Oct 23 22:06:03 2024,NaN,NaN,"{'qnt': [47], 'price': 5.71}","{'qnt': [40], 'price': 73.05}",NaN,NaN,"{'qnt': [11], 'price': 57.28}","{'qnt': [381.6529489935842], 'price': 39.7}","{'qnt': [1041.8968539622365], 'price': 15.03}","{'qnt': [2451.9684677637815], 'price': 41.48}","{'qnt': [1], 'price': 97.67}","{'qnt': [41], 'price': 99.45}",NaN
6,labp6aw837fsnzxgrd,Thu Oct 24 06:12:30 2024,"{'qnt': [33], 'price': 47.77}",NaN,"{'qnt': [32], 'price': 5.71}",NaN,NaN,NaN,"{'qnt': [625], 'price': 57.28}",NaN,"{'qnt': [600.2660933330224], 'price': 15.03}","{'qnt': [2174.6056069761953], 'price': 41.48}",NaN,NaN,"{'qnt': [429.78931768312174], 'price': 68.67}"
7,labp6aw80fyhiiipp51k,Thu Oct 24 02:55:33 2024,NaN,"{'qnt': [44], 'price': 48.35}","{'qnt': [34], 'price': 5.71}",NaN,NaN,"{'qnt': [12], 'price': 60.61}","{'qnt': [1056], 'price': 57.28}","{'qnt': [581.680609496261], 'price': 39.7}","{'qnt': [565.178563064884], 'price': 15.03}","{'qnt': [2207.5799004842866], 'price': 41.48}","{'qnt': [45], 'price': 97.67}","{'qnt': [25], 'price': 99.45}",NaN
8,labp6aw8pdpplarlhk7,Wed Oct 23 18:36:26 2024,NaN,"{'qnt': [7], 'price': 48.35}","{'qnt': [15], 'price': 5.71}",NaN,NaN,NaN,"{'qnt': [376], 'price': 57.28}","{'qnt': [962.9374971142064], 'price': 39.7}",NaN,"{'qnt': [2238.2638039102867], 'price': 41.48}","{'qnt': [11], 'price': 97.67}",NaN,NaN
9,labp6aw8t30qsei626,Wed Oct 23 21:38:40 2024,NaN,"{'qnt': [48], 'price': 48.35}","{'qnt': [27], 'price': 5.71}","{'qnt': [45], 'price': 73.05}",NaN,"{'qnt': [28], 'price': 60.61}","{'qnt': [610], 'price': 57.28}","{'qnt': [371.1562390052918], 'price': 39.7}",NaN,"{'qnt': [2291.9956971677916], 'price': 41.48}","{'qnt': [5], 'price': 97.67}","{'qnt': [32], 'price': 99.45}","{'qnt': [302.60742372446083], 'price': 68.67}"



product_database:


,prod_0,prod_2,prod_3,prod_4,prod_5,prod_6,prod_8,prod_9,prod_10,prod_15,prod_1,prod_7,prod_11
qnt,"[9, 22, 48, 33, 36, 1, 12, 13, 5, 44, 28, 14]","[25, 45, 44, 7, 48, 37, 25, 6, 7]","[16, 44, 47, 32, 34, 15, 27, 20, 36, 16, 2, 37...","[43, 42, 42, 7, 40, 45, 26, 42, 48, 9, 36, 15]","[1, 45, 5, 37, 8, 41, 22, 18]","[17, 46, 12, 28, 47, 11, 40, 6, 47]","[430, 402, 764, 880, 912, 11, 625, 1056, 376, ...","[809.6457464313798, 28.57175341014382, 1081.17...","[466.3152960854334, 893.6069315590282, 1081.34...","[1850.186334798945, 2426.738429691921, 2032.11...","[11, 6, 15, 1, 45, 11, 5, 16, 42, 31, 11, 5, 2...","[28, 26, 4, 21, 41, 25, 32, 19]","[907.3229546861892, 398.184360157671, 444.1580..."
price,47.77,48.35,5.71,73.05,38.61,60.61,57.28,39.7,15.03,41.48,97.67,99.45,68.67
next_week_price,47.77,48.35,5.71,73.05,38.61,60.61,57.28,39.7,15.03,41.48,97.67,99.45,68.67



Semana 2:

Json:


...

,prod_0,prod_2,prod_3,prod_4,prod_5,prod_6,prod_8,prod_9,prod_10,prod_15,prod_1,prod_7,prod_11
qnt,"[0, 20, 32, 34, 15, 37, 26, 35, 18, 10, 3, 28]","[32, 4, 19, 49, 29, 38, 18, 4, 12, 28, 38, 46,...","[7, 35, 23, 22, 42, 16, 19, 43, 11, 1, 17, 24,...","[23, 45, 31, 22, 23, 47, 37, 6, 22, 41]","[22, 19, 36, 36, 41, 1, 4, 49, 32, 31, 15, 8, ...","[8, 26, 4, 32, 7, 3, 39, 23, 20, 37, 9, 1, 26,...","[215, 707, 65, 361, 800, 488, 660, 736, 211, 2...","[26.594918587461457, 5.288894997974025, 27.298...","[123.50793182411205, 528.9693363421273, 817.56...","[2604.950666590065, 2464.7475053831768, 2430.2...","[29, 10, 47, 37, 9, 50, 15, 2, 20, 1]","[18, 15, 44, 39, 46, 30, 41, 4, 16, 20, 7, 7, ...","[133.54808118199296, 104.9314686416232, 689.83..."
price,356581.494751,361746.215008,9733.348621,546546.593956,63734.417945,453465.040651,428559.087373,297028.557414,112451.869469,310346.210618,730735.414665,254281.247431,175584.220795
next_week_price,534872.241681,542619.322507,14600.022931,819819.890933,95601.62691,680197.560743,642838.631059,445542.836122,168677.804204,465519.315928,1096103.121444,381421.871059,263376.331192



Semana 25:

Json:


,id,date,prod_0,prod_1,prod_2,prod_4,prod_5,prod_9,prod_11,prod_15,prod_3,prod_7,prod_8,prod_10,prod_6
0,labp6ctqdae2v7m3tx,Wed Apr 9 17:38:56 2025,"{'qnt': [17], 'price': 534872.2416807002}","{'qnt': [46], 'price': 1096103.1214438498}","{'qnt': [38], 'price': 542619.3225071158}","{'qnt': [42], 'price': 819819.8909333483}","{'qnt': [20], 'price': 95601.6269100002}","{'qnt': [4.10004804547035], 'price': 445542.83...","{'qnt': [442.33959914945706], 'price': 263376....","{'qnt': [2450.484536145639], 'price': 465519.3...",NaN,NaN,NaN,NaN,NaN
1,labp6ctqos5df0fyo38,Wed Apr 9 21:15:21 2025,NaN,NaN,NaN,NaN,"{'qnt': [26], 'price': 95601.6269100002}","{'qnt': [2.69183826329058], 'price': 445542.83...",NaN,"{'qnt': [2270.43934156096], 'price': 465519.31...","{'qnt': [21], 'price': 14600.022931134095}","{'qnt': [18], 'price': 381421.87105855916}","{'qnt': [37], 'price': 642838.6310589983}","{'qnt': [493.721440171984], 'price': 168677.80...",NaN
2,labp6ctq9nxu61szg6g,Wed Apr 9 20:15:20 2025,NaN,"{'qnt': [15], 'price': 1096103.1214438498}",NaN,"{'qnt': [9], 'price': 819819.8909333483}",NaN,"{'qnt': [1040.4637375641587], 'price': 445542....",NaN,"{'qnt': [2408.590791888705], 'price': 465519.3...","{'qnt': [16], 'price': 14600.022931134095}","{'qnt': [11], 'price': 381421.87105855916}","{'qnt': [31], 'price': 642838.6310589983}",NaN,NaN
3,labp6ctqi3z4txp0d0c,Wed Apr 9 18:21:57 2025,"{'qnt': [28], 'price': 534872.2416807002}","{'qnt': [20], 'price': 1096103.1214438498}","{'qnt': [5], 'price': 542619.3225071158}",NaN,NaN,"{'qnt': [877.8576003070798], 'price': 445542.8...",NaN,"{'qnt': [2454.8928522416863], 'price': 465519....","{'qnt': [29], 'price': 14600.022931134095}","{'qnt': [19], 'price': 381421.87105855916}","{'qnt': [9], 'price': 642838.6310589983}",NaN,"{'qnt': [10], 'price': 680197.5607430045}"
4,labp6ctq825sauzk00a,Thu Apr 10 00:40:21 2025,"{'qnt': [2], 'price': 534872.2416807002}","{'qnt': [24], 'price': 1096103.1214438498}",NaN,"{'qnt': [8], 'price': 819819.8909333483}","{'qnt': [15], 'price': 95601.6269100002}","{'qnt': [382.91350848493505], 'price': 445542....","{'qnt': [490.7511774550351], 'price': 263376.3...","{'qnt': [1940.445781741964], 'price': 465519.3...",NaN,"{'qnt': [38], 'price': 381421.87105855916}","{'qnt': [818], 'price': 642838.6310589983}","{'qnt': [654.4641190535075], 'price': 168677.8...",NaN
5,labp6ctqccvele3t4vw,Wed Apr 9 19:32:39 2025,NaN,"{'qnt': [9], 'price': 1096103.1214438498}",NaN,"{'qnt': [27], 'price': 819819.8909333483}","{'qnt': [8], 'price': 95601.6269100002}","{'qnt': [976.4786496873685], 'price': 445542.8...","{'qnt': [810.8655939937977], 'price': 263376.3...","{'qnt': [2485.582583923042], 'price': 465519.3...","{'qnt': [30], 'price': 14600.022931134095}",NaN,"{'qnt': [31], 'price': 642838.6310589983}","{'qnt': [881.3282722277088], 'price': 168677.8...",NaN
6,labp6ctq7pxwa0g0t26,Thu Apr 10 03:43:25 2025,"{'qnt': [25], 'price': 534872.2416807002}",NaN,NaN,"{'qnt': [10], 'price': 819819.8909333483}","{'qnt': [48], 'price': 95601.6269100002}","{'qnt': [513.5013609859717], 'price': 445542.8...",NaN,"{'qnt': [2546.5103749640066], 'price': 465519....","{'qnt': [11], 'price': 14600.022931134095}","{'qnt': [32], 'price': 381421.87105855916}","{'qnt': [48], 'price': 642838.6310589983}",NaN,NaN
7,labp6ctqiygdx8ivuwf,Wed Apr 9 19:45:22 2025,"{'qnt': [12], 'price': 534872.2416807002}",NaN,NaN,NaN,"{'qnt': [14], 'price': 95601.6269100002}","{'qnt': [385.81686769099497], 'price': 445542....","{'qnt': [915.2192665845338], 'price': 263376.3...","{'qnt': [2002.2953716941665], 'price': 465519....","{'qnt': [6], 'price': 14600.022931134095}",NaN,"{'qnt': [44], 'price': 642838.6310589983}",NaN,"{'qnt': [21], 'price': 680197.5607430045}"
8,labp6ctqnafkfqwhcaq,Thu Apr 10 08:37:57 2025,"{'qnt': [36], 'price': 534872.2416807002}","{'qnt': [19], 'price': 1096103.1214438498}",NaN,"{'qnt': [11], 'price': 819819.8909333483}",NaN,NaN,NaN,"{'qnt': [2626.266154809318], 'price': 465519.3...",NaN,"{'qnt': [42], 'price': 381421.87


product_database:


,prod_0,prod_2,prod_3,prod_4,prod_5,prod_6,prod_8,prod_9,prod_10,prod_15,prod_1,prod_7,prod_11
qnt,"[17, 28, 2, 25, 12, 36, 22, 30]","[38, 5, 17]","[21, 16, 29, 30, 11, 6, 39]","[42, 9, 8, 27, 10, 11, 19]","[20, 26, 15, 8, 48, 14, 37]","[10, 21, 8, 40]","[37, 31, 9, 818, 31, 48, 44, 943, 491, 651, 35]","[4.10004804547035, 2.69183826329058, 1040.4637...","[493.721440171984, 654.4641190535075, 881.3282...","[2450.484536145639, 2270.43934156096, 2408.590...","[46, 15, 20, 24, 9, 19, 12, 41, 39, 31]","[18, 11, 19, 38, 32, 42, 5]","[442.33959914945706, 490.7511774550351, 810.86..."
price,534872.241681,542619.322507,14600.022931,819819.890933,95601.62691,680197.560743,642838.631059,445542.836122,168677.804204,465519.315928,1096103.121444,381421.871059,263376.331192
next_week_price,802308.361852,813928.98072,21900.034382,1229729.80246,143402.440355,1020296.336221,964257.946588,668314.254182,253016.706306,698278.973891,1644154.682143,572132.806528,395064.496788
